In [1]:
#add folders to working path

In [2]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# add FastTensorlineComputation folder to current working path in order to access the functions
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/FastTensorlineComputation")

Wall time: 0 ns


# Import data

In [3]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../../data/ABC/ABC_steady.mat')

U = mat_file['u']
V = mat_file['v']
W = mat_file['w']
x = mat_file['x']
y = mat_file['y']
z = mat_file['z']

Wall time: 113 ms


# Data/Parameters for Dynamical System

In [4]:
import numpy as np

# number of cores to be used for parallel computing
Ncores = 18

# periodic boundary conditions
periodic_x = True
periodic_y = True
periodic_z = True
periodic = [periodic_x, periodic_y, periodic_z]

# unsteady velocity field
bool_unsteady = False

## compute meshgrid of dataset
X, Y, Z = np.meshgrid(x, y, z)

## resolution of meshgrid
dx_data = X[0,1,0]-X[0,0,0]
dy_data = Y[1,0,0]-Y[0,0,0]
dz_data = Z[0,0,1]-Z[0,0,0]

delta = [dx_data, dy_data, dz_data]

# Spatio-Temporal Domain of Dynamical System

In [5]:
%%time
from math import pi

# Time
t0 = 0
tN = 0
dt = 0.1

# store time in array
time = np.arange(t0, tN+dt, dt)

lenT = 10

# domain boundary (in degrees)
xmin = 0
xmax = 2*pi
ymin = 0
ymax = 2*pi
zmin = 0
zmax = 2*pi

# spacing of meshgrid (in degrees)
dx = 0.05
dy = 0.05
dz = 0.05

x_domain = np.linspace(xmin, xmax, int((xmax-xmin)/dx), endpoint = True)
y_domain = np.linspace(ymin, ymax, int((ymax-ymin)/dy), endpoint = True)
z_domain = np.linspace(ymin, ymax, int((zmax-zmin)/dz), endpoint = True)

X_domain, Y_domain, Z_domain = np.meshgrid(x_domain, y_domain, z_domain)

Wall time: 7.08 ms


# Interpolate Velocity

In order to evaluate the velocity field at arbitrary locations and times, we must interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". In order to favour a smooth velocity field, we interpolate the velocity field in space using a cubic interpolant.

In [6]:
%%time
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_steady

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0
W[np.isnan(W)] = 0

# Interpolate velocity data using linear spatial interpolation
Interpolant = interpolant_steady(X, Y, Z, U, V, W)

Interpolant_u = Interpolant[0]
Interpolant_v = Interpolant[1]
Interpolant_w = Interpolant[2]

Wall time: 44.4 ms


# Rate of Strain ($ \mathrm{S}(\mathbf{x})$)

In [7]:
%%time
# Import package for parallel computing
from joblib import delayed, Parallel

# Import function to compute gradient of velocity function
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# Import function to compute velocity
from ipynb.fs.defs.velocity import velocity

# Define ratio of auxiliary grid spacing vs original grid_spacing
aux_grid_ratio = .2 # [1/100, 1/5]
aux_grid = [aux_grid_ratio*dx, aux_grid_ratio*dy, aux_grid_ratio*dz]

# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny, 2)
x0 = X_domain.ravel()
y0 = Y_domain.ravel()
z0 = Z_domain.ravel()

# Split x0, y0, z0 into N (equal) batches for parallel computing
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

x0_batch = list(split(x0, Ncores))
y0_batch = list(split(y0, Ncores))
z0_batch = list(split(z0, Ncores))

def parallel_S(x0_batch, y0_batch, z0_batch):
    
    # Initial conditions
    X0 = np.array([x0_batch, y0_batch, z0_batch])
    
    # Compute gradient of velocity field
    grad_vel = gradient_velocity(time[0], X0, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, aux_grid)
    
    # Compute rate of strain tensor
    S = .5*(grad_vel+grad_vel.transpose(1,0,2))

    return S

results = Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel_S)(x0_batch[i], y0_batch[i], z0_batch[i]) for i in range(len(x0_batch)))

# Extract rate of strain tensor from results of parallel computation
S = results[0]

for i in range(1,len(results)):
    
    S = np.append(S, results[i], axis = 2)
    
# Reshape array from vectorized form to structured meshgrid
X0_gridded = x0.reshape((X_domain.shape[0], X_domain.shape[1]))
Y0_gridded = y0.reshape((X_domain.shape[0], X_domain.shape[1]))
S = S.reshape((3, 3, X_domain.shape[0], X_domain.shape[1]))

ValueError: not enough values to unpack (expected 7, got 5)

# Compute eigenvalues/eigenvectors of rate of strain

In [8]:
%%time
# Import package to compute eigenvalues of 3x3 matrix
from ipynb.fs.defs.eigen import eigen

def parallel_eigen(i):
    
    eigenvalue_parallel = np.zeros((X_domain.shape[1], X_domain.shape[2], 3, 3))*np.nan
    eigenvector_parallel = np.zeros((X_domain.shape[1], X_domain.shape[2], 3, 3))*np.nan
    
    for j in range(X_domain.shape[1]):
        
        for k in range(Z_domain.shape[2]):
            
            eigenvalue, eigenvector = eigen(S[i,j,k,:,:])
        
            eigenvalue_parallel[j,k,:,:] = np.diag(eigenvalue)
            eigenvector_parallel[j,k,:,:] = eigenvector
        
    return eigenvalue_parallel, eigenvector_parallel

results_eigen = np.array(Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel_eigen)(i) for i in tqdm(range(X_domain.shape[0]))))

#eigenvalues and eigenvectors
eigenvalues = results_eigen[:,0,:,:,:,:]
eigenvectors = results_eigen[:,1,:,:,:,:]
smax = eigenvalues[:,:,:,0,0]
smid = eigenvalues[:,:,:,1,1]
smin = eigenvalues[:,:,:,2,2]
eigenvector_max = eigenvectors[:,:,:,:,0]
eigenvector_mid = eigenvectors[:,:,:,:,1]
eigenvector_min = eigenvectors[:,:,:,:,2]

NameError: name 'tqdm' is not defined

## Compute Tensorfield properties

We now compute the properties of the rate of strain tensor 'S' such as the eigenvalues 's1', 's2', 's3' and eigenvectors 'eigenv1', 'eigenv2', 'eigenv3'. Furthermore, we also need the spatial derivatives of the elements of 'S'.

In [9]:
%%time
# Import (cubic) RectangularGridInterpolat from scipy
from scipy.interpolate import RegularGridInterpolator
        
# Compute gradients of elements of rate of strain tensor
S11 = np.nan_to_num(S[:,:,:,0,0], nan=0.0)
S12 = np.nan_to_num(S[:,:,:,0,1], nan=0.0)
S22 = np.nan_to_num(S[:,:,:,1,1], nan=0.0)
S33 = np.nan_to_num(S[:,:,:,2,2], nan=0.0)
S23 = np.nan_to_num(S[:,:,:,1,2], nan=0.0)
S13 = np.nan_to_num(S[:,:,:,0,2], nan=0.0)

# Interpolate elements of rate of strain tensor (per default: cubic)
interpS11 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S11)
interpS12 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S12)
interpS22 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S22)
interpS33 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S33)
interpS23 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S23)
interpS13 = RegularGridInterpolator((Y_domain[:,0,0], X_domain[0,:,0], Z_domain[0,0,:]), S13)

NameError: name 'S' is not defined

In [10]:
# import math tools
from math import atan, acos, asin, cos, sin, sqrt

# Import package to compute tensorline equation using ODE solver
from ipynb.fs.defs.tensorline_equation import _tensorline_equation

# Import package to compute tensorlines by re-orienting the vectorfield on the fly
from ipynb.fs.defs.RK4_tensorlines import _RK4_tensorlines

# Import package to solve ODE
from scipy.integrate import solve_ivp

# import plotting library
import matplotlib.pyplot as plt

# define integration interval of dummy variable 's'
t = [0, 10]
t_eval = np.linspace(t[0], t[-1], 1000)

# initial alignment of eigenvector
init_orientation = np.array([0, 0, 1])

# define eigenvector_field 
# {0: eigenvector field of maximum eigenvalue}
# {1: eigenvector field of middle eigenvalue}
# {2: eigenvector field of minimum eigenvalue}
idx_eigenvector = 0

counter = 0

def terminate_ODE(t, x_phi, interpS11, interpS12, interpS13, interpS22, interpS23, interpS33, aux_grid, idx_eigenvector):
                
    x = x_phi[0]%(2*pi)
    y = x_phi[1]%(2*pi)
    z = x_phi[2]%(2*pi)
    
    # compute gradient of velocity
    grad_vel = gradient_velocity(0, np.array([x, y, z]), X, Y, Z, Interpolant, periodic, bool_unsteady, aux_grid)
    
    # compute rate of strain
    S = RateStrain(grad_vel)
    
    # compute eigenvalues/eigenvectors of rate of strain
    lam = eigen(S)[0]
            
    event = abs(lam[0]-lam[1]) <= .01 or abs(lam[0]-lam[2]) <= .01 or abs(lam[1]-lam[2]) <= .01
                
    return 1-event
            
terminate_ODE.terminal = True

for i in range(0, X_domain.shape[0], 5):
    for j in range(0, Y_domain.shape[1], 5):
        for k in range(0, Z_domain.shape[2], 5):
            
            # initial position of particle
            xinit = np.array([X_domain[i, j, k], Y_domain[i, j, k], Z_domain[i, j, k]])
            
            # compute gradient of velocity
            grad_vel = gradient_velocity(t[0], xinit, X, Y, Z, Interpolant, periodic, bool_unsteady, aux_grid)
        
            # compute rate of strain
            S = RateStrain(grad_vel)
            
            # compute inital eigenvector orientation
            lamda, eigenv = eigen(S)
            initial_vector = eigenv[:,idx_eigenvector]
            print("initial particle position: ", xinit)
            if np.sign(init_orientation@initial_vector) < 0:
                initial_vector = -initial_vector
            
            if initial_vector[0] < 0 or initial_vector[1] < 0:
                phi0 = atan(initial_vector[1]/initial_vector[0])+pi
            else:
                phi0 = atan(initial_vector[1]/initial_vector[0])
            theta0 = acos(initial_vector[2])
                
            x0 = np.array([X_domain[i, j, k], Y_domain[i, j, k], Z_domain[i, j, k], phi0, theta0])
            
            # Compute tensorline by re-orienting vector field
            ds = t_eval[1]-t_eval[0]
            x_oriented = x0[:3].copy()
            x_prime = initial_vector.copy()
            trajectory_oriented = [[], [], []]
            trajectory_oriented[0].append(x_oriented[0])
            trajectory_oriented[1].append(x_oriented[1])
            trajectory_oriented[2].append(x_oriented[2])   
            phi_oriented, theta_oriented = [], []
            print("initial vector: ", x_prime)
            
            for idx_t in range(len(t_eval)):             
                x_oriented, x_prime = _RK4_tensorlines(X, Y, Z, x, x_prime, ds, periodic, interpS11, interpS12, interpS13, interpS22, interpS23, interpS33, idx_eigenvector)
                trajectory_oriented[0].append(x_oriented[0])
                trajectory_oriented[1].append(x_oriented[1])
                trajectory_oriented[2].append(x_oriented[2])   
                vx = x_prime[0]
                vy = x_prime[1]
                vz = x_prime[2]
                norm_v = sqrt(vx**2+vy**2+vz**2)
                if norm_v == 0:
                    phi_oriented.append(np.nan)
                    theta_oriented.append(np.nan)                   
                else:
                    if vx < 0:
                        phi_oriented.append((atan(vy/vx)+pi)%(2*pi))
                    else:
                        phi_oriented.append((atan(vy/vx))%(2*pi))
                    theta_oriented.append(acos(vz/norm_v))
            
            # solve ODE for tensorlines maximum eigenvector field of rate of strain
            solODE = solve_ivp(_tensorline_equation, t, x0, 'RK45', t_eval, rtol=1e-5, atol=1e-5, args=(interpS11, interpS12, interpS13, interpS22, interpS23, interpS33, aux_grid, idx_eigenvector))
                               
            x = solODE.y[0,:]%(2*pi)
            y = solODE.y[1,:]%(2*pi)
            z = solODE.y[2,:]%(2*pi)
            phi = solODE.y[3,:]%(2*pi)
            theta = solODE.y[4,:]%pi
            
            lam1_2, lam2_3, lam1_3 = [], [], []
            
            for ii in range(x.shape[0]):
                
                S11 = interpS11([y[ii], x[ii], z[ii]])[0]
                S12 = interpS12([y[ii], x[ii], z[ii]])[0]
                S22 = interpS22([y[ii], x[ii], z[ii]])[0]
                S13 = interpS13([y[ii], x[ii], z[ii]])[0]
                S23 = interpS23([y[ii], x[ii], z[ii]])[0]
                S33 = interpS33([y[ii], x[ii], z[ii]])[0]
    
                S21 = S12.copy()
                S31 = S13.copy()
                S32 = S23.copy()
    
                S = np.array([[S11, S12, S13], [S21, S22, S23], [S31, S32, S33]])
                lam = eigen(S)[0]
                lam1_2.append(abs(lam[0]-lam[1]))
                lam1_3.append(abs(lam[0]-lam[2]))
                lam2_3.append(abs(lam[1]-lam[2]))
                
            # plot trajectory in 3D
            fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (14, 10), dpi = 500)
            ax1.scatter(trajectory_oriented[0][::10], trajectory_oriented[1][::10], facecolor = "None", edgecolor = "r", label = r'manual', marker = "o", s = 50)
            ax2.scatter(trajectory_oriented[0][::10], trajectory_oriented[2][::10], facecolor = "None", edgecolor = "r", label = r'manual', marker = "o", s = 50)
            ax3.scatter(trajectory_oriented[1][::10], trajectory_oriented[2][::10], facecolor = "None", edgecolor = "r", label = r'manual', marker = "o", s = 50)
            ax1.scatter(x[::10], y[::10], marker = "^", label = r'automatic', s = 50, facecolor = "None", edgecolor = "b")
            ax1.set_xlabel("x")
            ax1.set_ylabel("y")
            ax1.set_title("projection onto x/y plane")
            ax2.set_xlabel("x")
            ax2.set_ylabel("z")
            ax2.scatter(x[::10], z[::10], marker = "^", label = r'automatic', s = 50, facecolor = "None", edgecolor = "b")
            ax2.set_title("projection onto x/z plane")
            ax3.scatter(y[::10], z[::10], marker = "^", label = r'automatic', s = 50, facecolor = "None", edgecolor = "b")
            ax3.set_xlabel("y")
            ax3.set_ylabel("z")
            ax3.set_title("projection onto y/z plane")
            ax4.scatter(phi, theta, label = r'automatic', marker = "^", s = 50, facecolor = "None", edgecolor = "b")
            ax4.scatter(phi_oriented, theta_oriented, label = r'manual', marker = "o", s = 50, facecolor = "None", edgecolor = "r")
            ax4.set_xlabel(r'$\phi$')
            ax4.set_ylabel(r'$\theta$')
            ax4.set_title(r'$ \theta - \phi $')
            ax1.scatter(trajectory_oriented[0][0], trajectory_oriented[1][0], c = "b", marker = "^", s = 100)
            ax2.scatter(trajectory_oriented[0][0], trajectory_oriented[2][0], c = "b", marker = "^", s = 100)
            ax3.scatter(trajectory_oriented[1][0], trajectory_oriented[2][0], c = "b", marker = "^", s = 100)
            plt.legend(loc="upper right", fontsize = 10)
            
            plt.savefig('./Fig/max/fig_'+np.str(int(counter)) + '.jpg')
            counter += 1
            plt.show()

ModuleNotFoundError: No module named 'ipynb.fs.defs.RK4_tensorlines'